<a href="https://colab.research.google.com/github/Hisyamm13/Modul10/blob/master/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community
!pip install replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 2.0 MB/s eta 0:00:00


In [2]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata

# Set the API token
api_token = userdata.get('REPLICATE_API_TOKEN')
os.environ["REPLICATE_API_TOKEN"] = api_token

# Model setup
model = "ibm-granite/granite-3.2-8b-instruct"

output = Replicate(
    model=model,
    replicate_api_token=api_token,
)


In [3]:
# Define the customer reviews
customer_reviews = [
"The battery lasts all day, but the phone gets hot during gaming.",
"The screen is too dim outdoors, but I love the colors indoors.",
"This phone is fast, but it keeps crashing when I open certain apps."
]

# Refine the prompt to include reviews
reviews_text = "\n".join([f"Review {i+1}: {review}" for i,
review in enumerate(customer_reviews)])

In [4]:
# Set model parameters for prompting with default values
parameters = {
"top_k": 0,
"top_p": 1.0,
"max_tokens": 256,
"min_tokens": 0,
"random_seed": None,
"repetition_penalty": 1.0,
"stopping_criteria": "length (256 tokens)",
"stopping_sequence": None
}

In [6]:
# Add initial prompt
refined_prompt = f""":
Classify these reviews as positive, negative, or mixed, and tag
relevant focus areas such as battery life, screen quality, or
performance

{reviews_text}
"""

# Invoke the model
response = output.invoke(refined_prompt, parameters=parameters)

# Print the response
print("Granite Model Refined Response:\n")
print(response)

Granite Model Refined Response:

1. Positive (with focus on battery life), Negative (with focus on performance during gaming)
2. Mixed (with focus on screen quality), Negative (with focus on screen quality outdoors)
3. Positive (with focus on performance), Negative (with focus on stability and app performance)


In [7]:
# Refine top_k model parameter value
parameters = {
"top_k": 5,
"top_p": 1.0,
"max_tokens": 256,
"min_tokens": 0,
"random_seed": None,
"repetition_penalty": 1.0,
"stopping_criteria": "length (256 tokens)",
"stopping_sequence": None
}

In [9]:
# Add initial prompt
refined_prompt = f""":
Classify these reviews as positive, negative, or mixed, and tag
relevant focus areas such as battery life, screen quality, or
performance

{reviews_text}
"""

# Invoke the model
response = output.invoke(refined_prompt, parameters=parameters)

# Print the response
print("Granite Model Refined Response:\n")
print(response)

Granite Model Refined Response:

1. Positive (Battery Life), Negative (Performance/Heat Management)
2. Mixed (Screen Quality), Negative (Outdoor Visibility)
3. Positive (Performance), Negative (Stability/App Compatibility)


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/Datasets/fashion_boutique.csv')

In [12]:
data.head()

,product_id,category,brand,season,size,color,original_price,markdown_percentage,current_price,purchase_date,stock_quantity,customer_rating,is_returned,return_reason
0,FB000001,Outerwear,Zara,Spring,XL,Red,196.01,0.0,196.01,2025-07-05,37,3.0,False,NaN
1,FB000002,Tops,Uniqlo,Winter,L,Pink,119.64,0.0,119.64,2025-08-06,2,2.5,False,NaN
2,FB000003,Accessories,Uniqlo,Winter,NaN,Black,33.80,0.0,33.80,2025-08-06,22,4.3,False,NaN
3,FB000004,Shoes,Uniqlo,Spring,XL,Black,75.36,0.0,75.36,2025-07-07,48,2.6,False,NaN
4,FB000005,Tops,Banana Republic,Winter,XL,Black,105.02,0.0,105.02,2025-08-06,10,NaN,False,NaN


In [18]:
def create_review_text(row):
    rating = row["customer_rating"]
    brand = row["brand"]
    category = row["category"]

    if pd.isna(rating):
        return f"I bought a {category} from {brand}, but I did not leave a review."

    # Kategori review berdasarkan rating
    if rating <= 2:
        review = f"The {category.lower()} from {brand} was disappointing and did not meet my expectations."
    elif rating <= 3.5:
        review = f"The {category.lower()} from {brand} was average, nothing special but usable."
    else:
        review = f"I really liked the {category.lower()} from {brand}, it met my expectations."

    # Tambahkan alasan return jika ada
    if row["is_returned"] and pd.notna(row["return_reason"]):
        review += f" However, I had to return it because {row['return_reason'].lower()}."

    return review

In [14]:
def generate_review_prompt(row):
    return {
        "instruction": "Classify the customer review as Positive, Neutral, or Negative based on the rating and review text.",
        "input": {
            "product_id": row["product_id"],
            "brand": row["brand"],
            "category": row["category"],
            "customer_rating": row["customer_rating"],
            "review_text": create_review_text(row)
        },
        "output": (
            "Negative" if row["customer_rating"] <= 2
            else "Neutral" if row["customer_rating"] <= 3.5
            else "Positive"
        )
    }

In [20]:
# Ambil 3 sample review
samples = data.sample(3, random_state=30)

# Generate prompt Granite format
prompts = [generate_review_prompt(row) for _, row in samples.iterrows()]

for p in prompts:
    print(p)
    print("="*100)


{'instruction': 'Classify the customer review as Positive, Neutral, or Negative based on the rating and review text.', 'input': {'product_id': 'FB001572', 'brand': 'Mango', 'category': 'Outerwear', 'customer_rating': 4.5, 'review_text': 'I really liked the outerwear from Mango, it met my expectations.'}, 'output': 'Positive'}
{'instruction': 'Classify the customer review as Positive, Neutral, or Negative based on the rating and review text.', 'input': {'product_id': 'FB001494', 'brand': 'Mango', 'category': 'Bottoms', 'customer_rating': 1.8, 'review_text': 'The bottoms from Mango was disappointing and did not meet my expectations.'}, 'output': 'Negative'}
{'instruction': 'Classify the customer review as Positive, Neutral, or Negative based on the rating and review text.', 'input': {'product_id': 'FB000122', 'brand': 'Gap', 'category': 'Bottoms', 'customer_rating': nan, 'review_text': 'I bought a Bottoms from Gap, but I did not leave a review.'}, 'output': 'Positive'}
